<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [3]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2
import re
import datetime
import dask

In [4]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [5]:
def list_endpoints(year,month,day):
    date_today = datetime.datetime(year, month, day)
    endpoints = []
    for i in range(1, 365):
        past_date = date_today - datetime.timedelta(days=i)
        year = past_date.strftime("%Y")
        month = past_date.strftime("%m")
        day = past_date.strftime("%d")
        endpoint = f"https://api.congress.gov/v3/congressional-record/?y={year}&m={month}&d={day}&api_key={key}"
        endpoints.append(endpoint)
    return endpoints

In [6]:
def find_pdf_links(endpoints_list):
  try:
    links_to_pdfs_with_matches = []
    for i in endpoints_list:
        response = requests.get(i)
        if response.json()['Results']['Issues'] != []:
          link_to_pdf_file = response.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
          pdf_file = requests.get(link_to_pdf_file)
          if pdf_file.content[-7:] == b'\n%%EOF\n':
            pdf_data = BytesIO(pdf_file.content)
            pdf_reader = PyPDF2.PdfReader(pdf_data)
            number_of_pages = len(pdf_reader.pages)
            for j in range(number_of_pages):
              page = pdf_reader.pages[j]
              text = page.extract_text()
              adam_smith_matches = re.findall(r'Adam\sSmith\W*', text)
              if adam_smith_matches != []:
                print(link_to_pdf_file)
                links_to_pdfs_with_matches.append(link_to_pdf_file)
    return links_to_pdfs_with_matches
  except Exception as e:
    print(e)
    return links_to_pdfs_with_matches

delayed_pdf_links = [dask.delayed(find_pdf_links)(list_endpoints(i,5,20)) for i in range(1998,2003)]

In [7]:
actual_pdf_links = dask.compute(delayed_pdf_links)
actual_pdf_links

https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf
https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf
https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf
https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf
https://www.congress.gov/105/crec/1998/05/14/144/61/CREC-1998-05-14.pdf
https://www.congress.gov/105/crec/1998/05/14/144/61/CREC-1998-05-14.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf
Missed the stop code in LZWDecode!
https://www.congress.gov/106/

https://www.congress.gov/107/crec/2002/04/09/CREC-2002-04-09.pdf
https://www.congress.gov/106/crec/2000/04/11/146/45/CREC-2000-04-11.pdf
https://www.congress.gov/107/crec/2002/03/20/CREC-2002-03-20.pdf
https://www.congress.gov/106/crec/2000/03/21/146/32/CREC-2000-03-21.pdf
https://www.congress.gov/107/crec/2002/02/28/CREC-2002-02-28.pdf
https://www.congress.gov/106/crec/2000/03/14/146/28/CREC-2000-03-14.pdf
https://www.congress.gov/106/crec/2000/03/14/146/28/CREC-2000-03-14.pdf
https://www.congress.gov/106/crec/2000/02/10/146/12/CREC-2000-02-10.pdf
https://www.congress.gov/107/crec/2002/01/24/CREC-2002-01-24.pdf


https://www.congress.gov/107/crec/2001/12/20/CREC-2001-12-20.pdf
https://www.congress.gov/106/crec/1999/11/17/145/163/CREC-1999-11-17.pdf
https://www.congress.gov/107/crec/2001/12/06/CREC-2001-12-06.pdf
https://www.congress.gov/106/crec/1999/11/09/145/157/CREC-1999-11-09.pdf
https://www.congress.gov/106/crec/1999/11/09/145/157/CREC-1999-11-09.pdf
https://www.congress.gov/107/crec/2001/11/30/CREC-2001-11-30.pdf
https://www.congress.gov/107/crec/2001/11/27/CREC-2001-11-27.pdf
https://www.congress.gov/107/crec/2001/11/14/CREC-2001-11-14.pdf
https://www.congress.gov/106/crec/1999/11/02/145/152/CREC-1999-11-02.pdf
https://www.congress.gov/106/crec/1999/11/01/145/151/CREC-1999-11-01.pdf
https://www.congress.gov/106/crec/1999/10/28/145/149/CREC-1999-10-28.pdf
https://www.congress.gov/107/crec/2001/10/30/CREC-2001-10-30.pdf
https://www.congress.gov/107/crec/2001/10/30/CREC-2001-10-30.pdf
https://www.congress.gov/106/crec/1999/10/27/145/148/CREC-1999-10-27.pdf
https://www.congress.gov/107/crec/

Missed the stop code in LZWDecode!


https://www.congress.gov/106/crec/2000/09/20/CREC-2000-09-20.pdf
https://www.congress.gov/106/crec/2000/09/14/CREC-2000-09-14.pdf
https://www.congress.gov/106/crec/2000/09/14/CREC-2000-09-14.pdf
https://www.congress.gov/106/crec/2000/09/08/CREC-2000-09-08.pdf


https://www.congress.gov/106/crec/2000/06/19/CREC-2000-06-19.pdf


https://www.congress.gov/106/crec/2000/06/15/146/75/CREC-2000-06-15.pdf


https://www.congress.gov/106/crec/2000/06/08/146/70/CREC-2000-06-08.pdf
https://www.congress.gov/106/crec/2000/06/08/146/70/CREC-2000-06-08.pdf


([['https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf',
   'https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf',
   'https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf',
   'https://www.congress.gov/105/crec/1998/05/19/144/64/CREC-1998-05-19.pdf',
   'https://www.congress.gov/105/crec/1998/05/14/144/61/CREC-1998-05-14.pdf',
   'https://www.congress.gov/105/crec/1998/05/14/144/61/CREC-1998-05-14.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-1998-05-13.pdf',
   'https://www.congress.gov/105/crec/1998/05/13/144/60/CREC-199